<a href="https://colab.research.google.com/github/deepampatel/randomnlp/blob/main/IndianLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Oct  3 11:02:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    22W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz

--2020-10-03 11:02:57--  https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz
Resolving oscar-public.huma-num.fr (oscar-public.huma-num.fr)... 134.158.33.192
Connecting to oscar-public.huma-num.fr (oscar-public.huma-num.fr)|134.158.33.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 313843639 (299M) [application/x-gzip]
Saving to: ‘mr_dedup.txt.gz’

mr_dedup.txt.gz     100%[===================>] 299.30M  11.1MB/s    in 29s     

2020-10-03 11:03:28 (10.2 MB/s) - ‘mr_dedup.txt.gz’ saved [313843639/313843639]



In [3]:
!ls

mr_dedup.txt.gz  sample_data


In [4]:
!gzip -d mr_dedup.txt.gz
!ls

mr_dedup.txt  sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
!wc -l mr_dedup.txt

1504068 mr_dedup.txt


In [6]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [7]:
!pip install tokenizers
!git clone https://github.com/huggingface/transformers
!pip install transformers/.


     |████████████████████████████████| 3.0MB 2.8MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 44854 (delta 0), reused 12 (delta 0), pack-reused 44842
Receiving objects: 100% (44854/44854), 32.18 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (31114/31114), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 10.8MB/s 
     |████████████████████████████████| 890kB 39.2MB/s 
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1082350 sha256=ba08081512f4db163054492ed7c052532b7ead842572754094d11676b79b5488
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_oa_m87/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c

In [8]:
!head -15  mr_dedup.txt

Home / motivational marathi story / उद्योजकता (Entrepreneurship) / यांना हे जमलय, तर आपल्याला का नाही जमणार ?
यापैकी कोणाचीही सुरुवात स्वप्नवत झाली नव्हती. प्रत्येकाने संघर्ष केला आहे. आपल्या वेड्या स्वप्नांचा पाठलाग केला आहे. हे सर्वजण यशस्वी होऊ शकले कारण त्यांनी आपली स्वप्न पूर्ण करण्यासाठी "पाहिलं पाऊल" उचलण्याचं धाडस केलं !
आपल्या स्वप्नांचा पाठपुरावा केला तर यश मिळतेच. योग्य संधी शोधता यायला हवी किंवा आलेल्या संधीचा योग्य फायदा घेता आला पाहिजे. या सर्वांना ते जमलं , मग आपल्याला का नाही जमणार ?
वेटर्स बिचारे मोठ्यांच्या उपस्थितीमुळे चेहर्‍यावर कौतुकाचे भाव आणून लहान मुलांना मोठ्यांइतकाच आदर देताना दिसले. वेटर्सच काय, तर कॅप्टन्ससुद्धा!
मनात कुठेतरी दुखावलो गेलो. आईवडिलांनी चुकून माकून वर्षातून एकदा हॉटेलमध्ये नेले तर वडिलांशी दुर्मीळरीत्या कुजबुजत्या स्वरात आईलासुद्धा बोलताना पाहिलेला मी!
ही पिढी बिघडली ती आधीच्या पिढीमुळे, हे सार्वकालीन लागू होणारे विधान मान्य करूनसुद्धा हे जरा अतीच होत आहे असे वाटत राहिले.
हो, असं दिसतं खरं आजूबाजुला. आपल्या मुलांनी कसं वागावं हे त्यांच्या आईवडि

In [9]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ["mr_dedup.txt"]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
#tokenizer.save_model(".", "marathibert")

In [10]:
!mkdir martbertochar
tokenizer.save_model("martbertochar")

['martbertochar/vocab.json', 'martbertochar/merges.txt']

In [11]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./martbertochar/vocab.json",
    "./martbertochar/merges.txt",
)

In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [19]:
op = tokenizer.encode("एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  ")

In [20]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()
import json

In [22]:
tokenizer.decode(op)

'<s>एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  </s>'

In [1]:
!head -n 800000 mr_dedup.txt > output1.txt

In [2]:
!wc -l output1.txt

800000 output1.txt


In [8]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [9]:
from transformers import RobertaTokenizerFast


tokenizer = RobertaTokenizerFast.from_pretrained("./martbertochar", max_len=512)

In [10]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [11]:
model.num_parameters()
# => 84 million parameters

83504416

In [7]:
%%time
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="output1.txt",
    block_size=128,
)

CPU times: user 7min 52s, sys: 16.1 s, total: 8min 8s
Wall time: 2min 35s


In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./martbertochar",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)


/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:267: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 4.7643681640625, 'learning_rate': 4.9500039996800255e-05, 'epoch': 0.03999680025597952, 'total_flos': 2052204527616000, 'step': 500}
{'loss': 4.2392138671875, 'learning_rate': 4.9000079993600514e-05, 'epoch': 0.07999360051195904, 'total_flos': 4104409055232000, 'step': 1000}
{'loss': 3.9138662109375, 'learning_rate': 4.8500119990400774e-05, 'epoch': 0.11999040076793857, 'total_flos': 6156613582848000, 'step': 1500}
{'loss': 3.5727275390625, 'learning_rate': 4.8000159987201026e-05, 'epoch': 0.15998720102391809, 'total_flos': 8208818110464000, 'step': 2000}
{'loss': 3.341453125, 'learning_rate': 4.750019998400128e-05, 'epoch': 0.1999840012798976, 'total_flos': 10261022638080000, 'step': 2500}
{'loss': 3.06745703125, 'learning_rate': 4.700023998080154e-05, 'epoch': 0.23998080153587714, 'total_flos': 12313227165696000, 'step': 3000}


In [ ]:
trainer.save_model("./martbertochar")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./martbertochar",
    tokenizer="./martbertochar"
)

In [ ]:
tokenizer.mask_token

'<mask>'

In [ ]:
fill_mask("अनेक सामाजिक चळवळीत, सत्याग्रहांमधे सहभाग घेतला.")

[{'score': 0.03394700214266777,
  'sequence': '<s>अनेक सामाजिक चळवळीत,०००मधे सहभाग घेतला.</s>',
  'token': 917,
  'token_str': 'à¥¦à¥¦à¥¦'},
 {'score': 0.013973863795399666,
  'sequence': '<s>अनेक सामाजिक चळवळीत, पणमधे सहभाग घेतला.</s>',
  'token': 753,
  'token_str': 'Ġà¤ªà¤£'},
 {'score': 0.01306256651878357,
  'sequence': '<s>अनेक सामाजिक चळवळीत,\xa0मधे सहभाग घेतला.</s>',
  'token': 353,
  'token_str': 'Âł'},
 {'score': 0.011251473799347878,
  'sequence': '<s>अनेक सामाजिक चळवळीत, मधे सहभाग घेतला.</s>',
  'token': 225,
  'token_str': 'Ġ'},
 {'score': 0.008268973790109158,
  'sequence': '<s>अनेक सामाजिक चळवळीत, ३मधे सहभाग घेतला.</s>',
  'token': 492,
  'token_str': 'Ġà¥©'}]